In [40]:
## Importing All the libraries
import pandas as pd 
import numpy as np
import seaborn as sns
import plotly.express as px

In [41]:
## Loading dataset
df = pd.read_csv('Bengaluru_House_Data.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [42]:
## Dimesions of the data
df.shape

(13320, 9)

It look like there are **13 columns** and **10300 rows** in the dataset


In [43]:
## Checking for null values
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

Well there are **5342** null values in the **total_sqft** column and **2297** null values in the **price** column.
So First I need to handle these null values.

Since Location has only 1 null value I will drop that row.

In [44]:
## Dropping Location's Null value
df.dropna(subset=['location'], inplace=True)
df.isnull().sum()

area_type          0
availability       0
location           0
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In my End Goal i don't need this **society** variable so I will drop that column.


In [45]:
# ## Dropping Society column
# df.drop(columns=['society'], inplace=True)
# df.shape

In [46]:
## handling Other columns with null values

#Filling size with most frequent value
df['size'].fillna(df['size'].mode()[0], inplace=True)


# Filling bath with most frequent value
df['bath'].fillna(df['bath'].mode()[0], inplace=True)

# Filling balcony with most frequent value
df['balcony'].fillna(df['balcony'].mode()[0], inplace=True)

# Filling society with most frequent value
df['society'].fillna(df['society'].mode()[0], inplace=True)

df.isnull().sum()



C:\Users\shane\AppData\Local\Temp\ipykernel_4428\2182470908.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\shane\AppData\Local\Temp\ipykernel_4428\2182470908.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

Pandas has an built-in function as **size** and so my data has. It can create trouble later so I will rename it.


In [47]:
## Renaming Size column as Sizes
df.rename(columns={'size': 'Sizes'}, inplace=True)
df.head()

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,GrrvaGr,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,GrrvaGr,1200,2.0,1.0,51.00


In [48]:
## Some statistical analysis
df.describe()

,bath,balcony,price
count,13319.000000,13319.000000,13319.000000
mean,2.688790,1.603349,112.567621
std,1.338801,0.803090,148.977089
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [49]:
## Checking Unique Values in baths
df.bath.unique()

array([ 2.,  5.,  3.,  4.,  6.,  1.,  9.,  8.,  7., 11., 10., 14., 27.,
       12., 16., 40., 15., 13., 18.])

Well **27, 40 , 18** in a normal house or room these much bathrooms are very unusual. So i am considering them as an outlier and removing them from the data.

In [50]:
## Removing those rows where bathroom is > 10
df = df[df.bath <= 10]
df.bath.unique()

array([ 2.,  5.,  3.,  4.,  6.,  1.,  9.,  8.,  7., 10.])

In [51]:
## Removing Those rows where total_sqft is less than 300 sqft

# Before i need to convert total_sqft into numeric data
df['total_sqft'] = pd.to_numeric(df['total_sqft'], errors='coerce')
df['total_sqft'] = df.total_sqft.fillna(df.total_sqft.mean())

df = df[df.total_sqft >= 300]

It does'nt Seems Like my data have any outliers.
At this point i am done with all the data Cleaning process. So Moving Further to some **EDA** Part for better understanding of the data.

# Exploratory Data Analysis

In [52]:
## Distribution of price
px.histogram(df, x='price', nbins=30).update_layout(bargap = 0.1)

In [53]:
## Distribution of sizes
px.histogram(df, x='Sizes', nbins=30).update_layout(bargap = 0.1)

In [54]:
## Distribution of Bathrooms
px.histogram(df, x='bath', nbins= df.bath.nunique()).update_layout(bargap = 0.1)

In [55]:
## Analysing Relation b/w Price and Bathrooms
px.violin(df, x='bath', y='price', color = 'bath').show()
print(df.bath.corr(df.price))

0.4712081386838736


It seems like **no. of bathrooms does'nt make that much of inpact on price.**

In [56]:
## Analysing relation b/w Price and Sizes
px.scatter(df, x='Sizes', y='price').show()

 Since having mostly Categorical data is creating issues in analysing there effect. We will deal with it later

Splitting data for training and testing purpose

In [57]:
## Importing train_test_split for splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['price'], test_size=0.2, random_state=42)

In [58]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10630, 9)
(2658, 9)
(10630,)
(2658,)


In [59]:
## Checking X_train test
X_train.head()

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony,price
6267,Super built-up Area,Ready To Move,Mysore Road,2 BHK,GrrvaGr,1003.0,2.0,1.0,43.0
7298,Plot Area,Ready To Move,Arekere,8 Bedroom,GrrvaGr,1200.0,8.0,2.0,225.0
9644,Super built-up Area,18-Dec,Electronics City Phase 1,3 BHK,Shitt S,1450.0,3.0,2.0,79.0
11087,Super built-up Area,Ready To Move,Doddakallasandra,2 BHK,GrrvaGr,1010.0,2.0,1.0,41.0
3436,Plot Area,Ready To Move,Anandapura,2 Bedroom,GrrvaGr,1200.0,2.0,1.0,58.0


Since in any machine learning model i can't directly put these categorical data so I will convert them into numerical data.

## Problem Statement
**1-**  area_type has 4 unique values.<br>
**2_** avaibility, location and Sizes has so many unique values. So i can't convert them all into different columns<br>
**3-** in total_sqft there are some values that are non numeric. So i will have to deal with them too.

In [60]:
## For my need i will not use OneHotEncoding for area_type. Since it can create complexity later.
from sklearn.preprocessing import MinMaxScaler
model = MinMaxScaler()

# For Training Data
train_area_type = X_train.groupby('area_type').price.mean()

# Applying Min Max Scalar 
train_area_enc = model.fit_transform(train_area_type.values.reshape(-1,1))

# For Training Data

train_society = X_train.groupby('society').price.mean()
train_society_enc = model.fit_transform(train_society.values.reshape(-1,1))




In [61]:
## Storing values of Each area_type and it's encoded value
area_type_dict = dict(zip(train_area_type.index, train_area_enc.flatten()))
area_type_dict

society_dict = dict(zip(train_society.index, train_society_enc.flatten()))
society_dict

{'3Codeli': 0.018598879460238676,
 '7 ise P': 0.015440859575859354,
 'A idse ': 0.015440859575859354,
 'A rtsai': 0.02398608985123869,
 'ACersd ': 0.039590423397583556,
 'ACiteda': 0.01636968895361798,
 'ACopsva': 0.010518063873738652,
 'AFe 3Ph': 0.03401744713103182,
 'AGoutLa': 0.08974720979654922,
 'AHavenc': 0.10832379735172169,
 'AHodsWo': 0.01767005008248005,
 'ALoraAu': 0.006152565798273121,
 'ALtonea': 0.014194989770492456,
 'AMAKS O': 0.008010224553790368,
 'AMaveon': 0.013906433443802109,
 'APncyla': 0.020790916791749027,
 'APuraen': 0.14547697246206662,
 'APusenc': 0.03401744713103182,
 'ARaveSa': 0.013583200820342107,
 'AReenui': 0.02287149459792834,
 'ARestco': 0.019527708837997298,
 'ARineha': 0.021013835842411094,
 'ARipsul': 0.013037049146220038,
 'ARnamru': 0.01242216409814383,
 'ARncy R': 0.02472915335344559,
 'ARngsd ': 0.011226946454844032,
 'ARntyne': 0.023119182431997308,
 'ARomee ': 0.010425180935962789,
 'ARureig': 0.018688666300088676,
 'ARvasV ': 0.01955557371

In [62]:
## Mapping these same values in Test set
X_test['area_type'] = X_test['area_type'].map(area_type_dict)
X_test

X_test['society'] = X_test['society'].map(society_dict).fillna(0)
X_test

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony,price
2368,1.000000,Ready To Move,Bileshivale,5 Bedroom,0.000000,6040.000000,4.0,2.0,170.00
13139,0.004600,Ready To Move,Nagarbhavi,2 BHK,0.043894,1225.000000,2.0,3.0,58.00
4148,0.004600,Ready To Move,Bisuvanahalli,3 BHK,0.012183,1180.000000,2.0,1.0,46.00
387,0.119961,Ready To Move,Vijayanagar,3 BHK,0.043894,1580.000000,2.0,0.0,134.00
2080,0.004600,Ready To Move,Bannerghatta Road,3 BHK,0.000000,1760.000000,3.0,2.0,67.00
...,...,...,...,...,...,...,...,...,...
8023,0.119961,Ready To Move,Devanahalli,4 Bedroom,0.000000,1551.634397,4.0,0.0,393.50
4760,0.004600,Ready To Move,Kaggadasapura,2 BHK,0.013583,1035.000000,2.0,2.0,45.50
4763,0.004600,21-Feb,Kanakpura Road,3 BHK,0.013439,1100.000000,3.0,1.0,52.97
5020,0.004600,Ready To Move,Richards Town,4 BHK,0.043894,3600.000000,4.0,3.0,450.00


In [63]:
## Similarly For availability, location and sizes columns

# For Avaibility column

train_availability = X_train.groupby('availability').price.mean()

train_availability_enc = model.fit_transform(train_availability.values.reshape(-1,1))

availability_dict = dict(zip(train_availability.index, train_availability_enc.flatten()))


# For Location Column

train_location = X_train.groupby('location').price.mean()

train_location_enc = model.fit_transform(train_location.values.reshape(-1,1))

location_dict = dict(zip(train_location.index, train_location_enc.flatten()))

# Same for Sizes Column

train_sizes = X_train.groupby('Sizes').price.mean()

train_sizes_enc = model.fit_transform(train_sizes.values.reshape(-1,1))

sizes_dict = dict(zip(train_sizes.index, train_sizes_enc.flatten()))



In [64]:
## setting same values in Test Set

default = 0   # If any value does'nt match with train set then it will be replaced by 0

X_test['availability'] = X_test['availability'].map(availability_dict).fillna(default)

X_test['location'] = X_test['location'].map(location_dict).fillna(default)

X_test['Sizes'] = X_test['Sizes'].map(sizes_dict).fillna(default)

X_test.head()

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony,price
2368,1.000000,0.219692,0.145751,0.535064,0.000000,6040.0,4.0,2.0,170.0
13139,0.004600,0.219692,0.069086,0.065802,0.043894,1225.0,2.0,3.0,58.0
4148,0.004600,0.219692,0.017910,0.176381,0.012183,1180.0,2.0,1.0,46.0
387,0.119961,0.219692,0.071519,0.176381,0.043894,1580.0,2.0,0.0,134.0
2080,0.004600,0.219692,0.059728,0.176381,0.000000,1760.0,3.0,2.0,67.0


In [65]:
## Assigning Values in Training Dataset
X_train['area_type'] = X_train['area_type'].map(area_type_dict).fillna(default)

X_train['availability'] = X_train['availability'].map(availability_dict).fillna(default)

X_train['location'] = X_train['location'].map(location_dict).fillna(default)

X_train['Sizes'] = X_train['Sizes'].map(sizes_dict).fillna(default)

X_train['society'] = X_train['society'].map(society_dict).fillna(default)


In [66]:
X_train

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony,price
6267,0.004600,0.219692,0.035142,0.065802,0.043894,1003.0,2.0,1.0,43.0
7298,1.000000,0.219692,0.062359,0.431876,0.043894,1200.0,8.0,2.0,225.0
9644,0.004600,0.167255,0.027097,0.176381,0.020073,1450.0,3.0,2.0,79.0
11087,0.004600,0.219692,0.029584,0.065802,0.043894,1010.0,2.0,1.0,41.0
3436,1.000000,0.219692,0.028668,0.151365,0.043894,1200.0,2.0,1.0,58.0
...,...,...,...,...,...,...,...,...,...
11994,0.119961,0.219692,0.049926,0.176381,0.048693,1610.0,2.0,3.0,119.0
5210,0.119961,0.219692,0.038543,0.065802,0.026370,1340.0,2.0,2.0,60.0
5409,0.004600,0.219692,0.045159,0.176381,0.029270,2017.0,2.0,2.0,125.0
863,0.119961,0.219692,0.034650,0.065802,0.016184,1200.0,2.0,2.0,52.0


Here we can see that there are some non numeric values in sqft. So we would have to either replace or delete those rows.

In [67]:
## Assigning new values to total_sqft

# For Training Data
X_train['new_total_sqft'] = pd.to_numeric(X_train['total_sqft'], errors = 'coerce')

X_train.new_total_sqft.fillna(X_train.new_total_sqft.mean(), inplace = True)

X_train.new_total_sqft.isnull().sum()

# For Test Data
X_test['new_total_sqft'] = pd.to_numeric(X_test['total_sqft'], errors = 'coerce')

X_test.new_total_sqft.fillna(X_test.new_total_sqft.mean(), inplace = True)

X_test.new_total_sqft.isnull().sum()

C:\Users\shane\AppData\Local\Temp\ipykernel_4428\2596152884.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\shane\AppData\Local\Temp\ipykernel_4428\2596152884.py:13: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

0

At this time we are now able to scale down all numeric columns like **(total_sqft, bath, balcony)**

In [68]:
## Scaling All Numeric Columns
# For Training
numeric_cols = ['total_sqft', 'bath', 'balcony']

scalar = MinMaxScaler()

scalar.fit(X_train[numeric_cols])

## Transforming Both Training and test dataset
X_train[numeric_cols] = scalar.transform(X_train[numeric_cols])

X_test[numeric_cols] = scalar.transform(X_test[numeric_cols])



In [69]:
X_train.drop('new_total_sqft', axis = 1, inplace = True)
X_test.drop('new_total_sqft', axis = 1, inplace = True)

At this point we can remove **Price** column from both test and training dataset.


In [70]:
## Removing Price Column
X_train.drop('price', axis = 1, inplace = True)

X_test.drop('price', axis = 1, inplace = True)

X_train.head()

,area_type,availability,location,Sizes,society,total_sqft,bath,balcony
6267,0.0046,0.219692,0.035142,0.065802,0.043894,0.013527,0.111111,0.333333
7298,1.0000,0.219692,0.062359,0.431876,0.043894,0.017317,0.777778,0.666667
9644,0.0046,0.167255,0.027097,0.176381,0.020073,0.022127,0.222222,0.666667
11087,0.0046,0.219692,0.029584,0.065802,0.043894,0.013661,0.111111,0.333333
3436,1.0000,0.219692,0.028668,0.151365,0.043894,0.017317,0.111111,0.333333


In [71]:
## Now we can train our data

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
## Now we can evaluate our model
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2_score(y_test, y_pred)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(y_test, y_pred)}')

Mean Squared Error: 8401.417261997876
R2 Score: 0.534406606880982
Root Mean Squared Error: 91.65924537109105


In [72]:
## Creating a dict to store the statistices of numerical data ( will use later for converting any numeric value into min max scalar of same format )
max = scalar.data_max_

min = scalar.data_min_

val_dic_num = {
    'total_sqft': [max[0], min[0]],
    'bath': [max[1], min[1]],
    'balcony': [max[2], min[2]]
}
val_dic_num

{'total_sqft': [52272.0, 300.0], 'bath': [10.0, 1.0], 'balcony': [3.0, 0.0]}

In [73]:
## Dictiorney For storing all the dictionries for a compacted dictioreny
variables_dict = area_type_dict | availability_dict | location_dict | sizes_dict | val_dic_num | society_dict
variables_dict

{'Built-up  Area': 0.11996119172013464,
 'Carpet  Area': 0.0,
 'Plot  Area': 1.0,
 'Super built-up  Area': 0.004600468799626167,
 '14-Jul': 0.01359013200605931,
 '15-Aug': 0.06636370194041695,
 '15-Dec': 0.26256942941643224,
 '15-Nov': 0.24814253769025463,
 '15-Oct': 0.20774724085695734,
 '16-Dec': 0.3202769963211426,
 '16-Jul': 0.2596840510711967,
 '16-Mar': 0.43857750847579885,
 '16-Nov': 0.0,
 '16-Oct': 0.07501983697612351,
 '16-Sep': 0.6088148308446946,
 '17-Apr': 0.10176729423645679,
 '17-Aug': 0.10961552333549736,
 '17-Dec': 0.2548056243504559,
 '17-Feb': 0.014426891726177601,
 '17-Jan': 0.2885378345235519,
 '17-Jul': 0.3241758638101421,
 '17-Jun': 0.10117739466365307,
 '17-Mar': 0.21027194690903844,
 '17-May': 0.18466421409507322,
 '17-Nov': 0.2810445069609752,
 '17-Oct': 0.07367728740489919,
 '17-Sep': 0.06353603116208613,
 '18-Apr': 0.17487890761713165,
 '18-Aug': 0.09839251849318094,
 '18-Dec': 0.16725538608691004,
 '18-Feb': 0.24892772894633042,
 '18-Jan': 0.3932357915153846

In [76]:
import pickle

# Assuming df is already defined
columns = df.columns
unique_val_dic = {}
for i in columns:
    unique_val_dic[i] = df[i].unique().tolist()

# Save the dictionary to a pickle file
with open('unique_val_dic.pkl', 'wb') as f:
    pickle.dump(unique_val_dic, f)

In [2]:
## Creating GUI 
from customtkinter import *
import pickle
import joblib

# Load the trained model
model = joblib.load('model.pkl')

# Load the dictionary
with open('variables_dict.pkl', 'rb') as f:
    variables_dict = pickle.load(f)

# Loading Unique_values Dictiorney
with open('unique_val_dic.pkl', 'rb') as f:
    unique_val_dic = pickle.load(f)

def check_fields():
    if (area_dropdown.get() and availability_dropdown.get() and location_dropdown.get() and 
        sizes_dropdown.get() and society_dropdown.get() and SQFT_entry.get() and 
        bath_dropdown.get() and balcony_dropdown.get()):
        submit_button.configure(state='normal', fg_color='blue')
    else:
        submit_button.configure(state='disabled', fg_color='gray')

def on_submit():
    result = []
    for widget in result_frame.winfo_children():
        widget.destroy()
      
    area_type = area_dropdown.get()
    avilability = availability_dropdown.get()
    location = location_dropdown.get()
    sizes = sizes_dropdown.get()
    society = society_dropdown.get()
    total_sqft = SQFT_entry.get()
    bath = bath_dropdown.get()
    balcony = balcony_dropdown.get()
    total_sqft = int(total_sqft)
    bath = float(bath)
    balcony = float(balcony)
    
    input_data = [area_type, avilability, location, sizes, society, total_sqft, bath, balcony]

    for i in range(len(input_data)):
        if i == 5:  # total_sqft
            X = input_data[i]
            enc_X = (X - variables_dict['total_sqft'][1]) / (variables_dict['total_sqft'][0] - variables_dict['total_sqft'][1])
            result.append(enc_X)
        elif i == 6:  # bath
            X = input_data[i]
            enc_X = (X - variables_dict['bath'][1]) / (variables_dict['bath'][0] - variables_dict['bath'][1])
            result.append(enc_X)
        elif i == 7:  # balcony
            X = input_data[i]
            enc_X = (X - variables_dict['balcony'][1]) / (variables_dict['balcony'][0] - variables_dict['balcony'][1])
            result.append(enc_X)
        else:
            result.append(variables_dict.get(input_data[i], 0))

    predicted_value = model.predict([result])[0]
    if predicted_value < 0:
        CTkLabel(result_frame, text=f'The model has been trained in a way that it will through negative value which is not true. Please Increase the Total SQFT', text_color='Red', font=('Candara', 15, 'bold')).pack(pady=10)
    else:
        CTkLabel(result_frame, text=f'The Estimated Price Should be {predicted_value}', text_color='Green', font=('Candara', 15, 'bold')).pack(pady=10)

root = CTk()

## Setting the geometry of the window
root.geometry('1000x100')

## Adding Heading to my GUI
header = CTkLabel(root, text='Bangalore House Prediction', text_color='#080A07', font=('Candara', 30, 'bold')).pack(pady=20)
subheader = CTkLabel(root, text='By Shane Rahman', text_color='#080A07', font=('Candara', 15)).pack()

## Creating Div Sections For all the variables
frames = ['area', 'availability', 'location', 'sizes', 'society', 'totalSqft', 'bath', 'balcony']
for frame in frames:
    exec(f"{frame}_frame = CTkFrame(root); {frame}_frame.pack(pady=10)")

## Adding Elements
#area_type dropdown
area_header = CTkLabel(area_frame, text='Choose The Area Type', text_color='#080A07', font=('Candara', 15)).pack()
area_dropdown = CTkComboBox(area_frame, values=unique_val_dic['area_type'], width=200)
area_dropdown.pack()

#Avilability Dropdown
availability_header = CTkLabel(availability_frame, text='Choose The Avilability', text_color='#080A07', font=('Candara', 15)).pack()
availability_dropdown = CTkComboBox(availability_frame, values=unique_val_dic['availability'], width=200)
availability_dropdown.pack()

#Location Dropdown
location_header = CTkLabel(location_frame, text='Choose The Location', text_color='#080A07', font=('Candara', 15)).pack()
location_dropdown = CTkComboBox(location_frame, values=unique_val_dic['location'], width=200)
location_dropdown.pack()

# Sizes Dropdown
sizes_header = CTkLabel(sizes_frame, text='Choose The Size', text_color='#080A07', font=('Candara', 15)).pack()
sizes_dropdown = CTkComboBox(sizes_frame, values=unique_val_dic['Sizes'], width=200)
sizes_dropdown.pack()

#society Dropdown
society_header = CTkLabel(society_frame, text='Choose The Society', text_color='#080A07', font=('Candara', 15)).pack()
society_dropdown = CTkComboBox(society_frame, values=unique_val_dic['society'], width=200)
society_dropdown.pack()

# Total SQFT Entry Box
SQFT_header = CTkLabel(totalSqft_frame, text='Enter the Total_SQFT', text_color='#080A07', font=('Candara', 15)).pack()
SQFT_entry = CTkEntry(totalSqft_frame, width=200)
SQFT_entry.pack()
SQFT_entry.bind('<KeyRelease>', lambda event: check_fields())

# Bath Dropdown
bath_header = CTkLabel(bath_frame, text='Choose The No of Bathrooms', text_color='#080A07', font=('Candara', 15)).pack()
bath_dropdown = CTkComboBox(bath_frame, values=[str(value) for value in unique_val_dic['bath']], width=200)
bath_dropdown.pack()

#Balcony Dropdown
balcony_header = CTkLabel(balcony_frame, text='Choose The No. of Balcony', text_color='#080A07', font=('Candara', 15)).pack()
balcony_dropdown = CTkComboBox(balcony_frame, values=[str(value) for value in unique_val_dic['balcony']], width=200)
balcony_dropdown.pack()

## Adding Estimate Value Button
submit_button = CTkButton(root, command=on_submit, text='Check Price', state='disabled', fg_color='gray')
submit_button.pack()

# Frame for output
result_frame = CTkFrame(root, width=600, height=10)
result_frame.pack(pady=10)

root.mainloop()


c:\Users\shane\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
